# Example of using the PyRadiomics toolbox in Python

First, import some built-in Python modules needed to get our testing data.
Second, import the toolbox, only the `featureextractor` is needed, this module handles the interaction with other parts of the toolbox.

In [1]:
from __future__ import print_function
import six
import os  # needed navigate the system to get the input data

import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

### Setting up data

Test cases can be downloaded to temporary files. This is handled by the `radiomics.getTestCase()` function, which checks if the requested test case is available and if not, downloads it. It returns a tuple with the location of the image and mask of the requested test case, or (None, None) if it fails.

Alternatively, if the data is available somewhere locally, this directory can be passed as a second argument to `radiomics.getTestCase()`. If that directory does not exist or does not contain the testcase, functionality reverts to default and tries to download the test data.

If getting the test case fails, PyRadiomics will log an error explaining the cause.

In [2]:
# Get the testCase
imagePath, maskPath = radiomics.getTestCase('brain1')

if imagePath is None or maskPath is None:  # Something went wrong, in this case PyRadiomics will also log an error
    raise Exception('Error getting testcase!')  # Raise exception to prevent cells below from running in case of "run all"

# Additonally, store the location of the example parameter file, stored in \pyradiomics\examples/exampleSettings
paramPath = os.path.join('..', 'examples', 'exampleSettings', 'Params.yaml')
print('Parameter file, absolute path:', os.path.abspath(paramPath))

Parameter file, absolute path: /project/6003292/marafath/pyradiomics/notebooks/examples/exampleSettings/Params.yaml


#### COVID Data and Mask paths

In [3]:
imagePath = '/home/marafath/scratch/iran_organized_data2/1924519_1/03/image.nii.gz'
maskPath = '/home/marafath/scratch/iran_organized_data2/1924519_1/03/infection.nii.gz'

### Instantiating the extractor

Now that we have our input, we need to define the parameters and instantiate the extractor.
For this there are three possibilities:

1. Use defaults, don't define custom settings

2. Define parameters in a dictionary, control filters and features after initialisation

3. Use a parameter file

##### Method 1, use defaults

In [4]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor()

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}


##### Method 2, hard-coded settings:

In [13]:
# First define the settings
settings = {}
settings['binWidth'] = 20
settings['sigma'] = [1, 2, 3]

# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)  # ** 'unpacks' the dictionary in the function call

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)  # Still the default parameters
print('Enabled features:\n\t', extractor.enabledFeatures)  # Still the default parameters

Extraction parameters:
	 {'distances': [1], 'additionalInfo': True, 'force2D': False, 'interpolator': 'sitkBSpline', 'resampledPixelSpacing': None, 'label': 1, 'normalizeScale': 1, 'normalize': False, 'force2Ddimension': 0, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 20, 'minimumROIDimensions': 2, 'preCrop': False, 'resegmentRange': None, 'sigma': [1, 2, 3], 'padDistance': 5}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'gldm': [], 'glcm': [], 'shape': [], 'firstorder': [], 'glrlm': [], 'glszm': [], 'ngtdm': []}


In [14]:
# This cell is equivalent to the previous cell
extractor = featureextractor.RadiomicsFeatureExtractor(binWidth=20, sigma=[1, 2, 3])  # Equivalent of code above

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)  # Still the default parameters
print('Enabled features:\n\t', extractor.enabledFeatures)  # Still the default parameters

Extraction parameters:
	 {'distances': [1], 'additionalInfo': True, 'force2D': False, 'interpolator': 'sitkBSpline', 'resampledPixelSpacing': None, 'label': 1, 'normalizeScale': 1, 'normalize': False, 'force2Ddimension': 0, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 20, 'minimumROIDimensions': 2, 'preCrop': False, 'resegmentRange': None, 'sigma': [1, 2, 3], 'padDistance': 5}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'gldm': [], 'glcm': [], 'shape': [], 'firstorder': [], 'glrlm': [], 'glszm': [], 'ngtdm': []}


In [15]:
# Enable a filter (in addition to the 'Original' filter already enabled)
extractor.enableImageTypeByName('LoG')
print('')
print('Enabled filters:\n\t', extractor.enabledImagetypes)

# Disable all feature classes, save firstorder
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
print('')
print('Enabled features:\n\t', extractor.enabledFeatures)

# Specify some additional features in the GLCM feature class
extractor.enableFeaturesByName(glcm=['Autocorrelation', 'Homogeneity1', 'SumSquares'])
print('')
print('Enabled features:\n\t', extractor.enabledFeatures)


Enabled filters:
	 {'Original': {}, 'LoG': {}}

Enabled features:
	 {'firstorder': []}

Enabled features:
	 {'firstorder': [], 'glcm': ['Autocorrelation', 'Homogeneity1', 'SumSquares']}


##### Method 3, using a parameter file

In [16]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'distances': [1], 'additionalInfo': True, 'weightingNorm': None, 'force2D': False, 'interpolator': 'sitkBSpline', 'resampledPixelSpacing': None, 'label': 1, 'normalizeScale': 1, 'normalize': False, 'force2Ddimension': 0, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 25, 'minimumROIDimensions': 2, 'preCrop': False, 'resegmentRange': None, 'padDistance': 5}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares'], 'shape': None, 'gldm': None, 'firstorder': [], 'glrlm': None, 'glszm': None}


### Extract features

Now that we have our extractor set up with the correct parameters, we can start extracting features:

In [5]:
result = extractor.execute(imagePath, maskPath)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [6]:
print(len(result))

129


In [7]:
print('Result type:', type(result))  # result is returned in a Python ordered dictionary)
print('')
print('Calculated features')
for key, value in six.iteritems(result):
    print('\t', key, ':', value)

Result type: <class 'collections.OrderedDict'>

Calculated features
	 diagnostics_Versions_PyRadiomics : v3.0.post9+g51639ec
	 diagnostics_Versions_Numpy : 1.18.1
	 diagnostics_Versions_SimpleITK : 1.2.2
	 diagnostics_Versions_PyWavelet : 0.5.2
	 diagnostics_Versions_Python : 3.7.4
	 diagnostics_Configuration_Settings : {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
	 diagnostics_Configuration_EnabledImageTypes : {'Original': {}}
	 diagnostics_Image-original_Hash : 544cea5bf292e31f73e61bc039cd6cfbe6c9bb4b
	 diagnostics_Image-original_Dimensionality : 3D
	 diagnostics_Image-original_Spacing : (1.0, 1.0, 1.0)
	 diagnostics_Image-original_Size : (161, 161, 96)
	 diagnostics_Image-original_Mean : 0.319159986

In [1]:
import csv

filename = "/home/marafath/scratch/Age_sex.csv"
rows = [] 

with open(filename, 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 
  
    # get total number of rows 
    print("Total no. of rows: %d"%(csvreader.line_num))

Total no. of rows: 590


In [7]:
print(rows[1])

['1919949_0', '076Y', 'F']


In [3]:
print(len(rows))

590


In [13]:
data_dir = '/home/marafath/scratch/iran_organized_data2'
import os

for patient in os.listdir(data_dir):
    rad = []
    for pat in range(0,len(rows)):
        if patient == rows[pat][0]:
            #print(rows[pat])
            
            if rows[pat][1] == '0':
                age = 50
            else:
                age = rows[pat][1]
                age = age[1:3]
                age = int(age)
            rad.append(age/100)
            
            if rows[pat][2] == ' ':
                rad.append(1)
                rad.append(0)
            elif rows[pat][2] == 'M':
                rad.append(1)
                rad.append(0)
            else:    
                rad.append(0)
                rad.append(1)
    
            #print(rad)
            break

['1699177_0', '039Y', 'F']
[0.39, 0, 1]
['1742462_0', '070Y', 'F']
[0.7, 0, 1]
['1234641_1', '068Y', 'M']
[0.68, 1, 0]
['1738796_0', '063Y', 'M']
[0.63, 1, 0]
['1726984_0', '085Y', 'F']
[0.85, 0, 1]
['1234713_1', '042Y', 'M']
[0.42, 1, 0]
['1726936_0', '068Y', 'F']
[0.68, 0, 1]
['1936943_1', '079Y', 'M']
[0.79, 1, 0]
['1327882_0', '040Y', 'M']
[0.4, 1, 0]
['1234712_1', '028Y', 'M']
[0.28, 1, 0]
['1325603_0', '033Y', 'M']
[0.33, 1, 0]
['1777683_0', '050Y', 'F']
[0.5, 0, 1]
['1333660_0', '061Y', 'M']
[0.61, 1, 0]
['1234695_1', '060Y', 'M']
[0.6, 1, 0]
['MEH_193_1', '035Y', 'F']
[0.35, 0, 1]
['1892925_0', '062Y', 'M']
[0.62, 1, 0]
['1931027_1', '070Y', 'M']
[0.7, 1, 0]
['1234711_1', '026Y', 'F']
[0.26, 0, 1]
['1866342_0', '074Y', 'F']
[0.74, 0, 1]
['1604778_0', '051Y', 'F']
[0.51, 0, 1]
['1761536_0', '083Y', 'M']
[0.83, 1, 0]
['1944955_1', '056Y', 'M']
[0.56, 1, 0]
['1929609_1', '058Y', 'F']
[0.58, 0, 1]
['1888799_0', '036Y', 'M']
[0.36, 1, 0]
['1234620_0', '039Y', 'M']
[0.39, 1, 0]
['132